In [1]:
import pickle

from support import *

In [2]:
pack = pickle.load(open('results.pkl', 'rb'))

In [4]:
pack

[TyvResult(broad_words=['like', 'think', 'go', 'look', 'him', 'here', 'after', 'call', 'next', 'pay', 'enough', 'hope', 'close', 'tomorrow', 'box', 'hair', 'clean', 'corner', 'skill', 'twice', 'bright', 'clever', 'moon', 'soul', 'hint', 'ferry', 'bat', 'dairy', 'grumble', 'wag', 'throttle', 'stubble', 'pittance', 'midget', 'discomfit', 'prig', 'soothsayer', 'imbroglio', 'legerdemain', 'uxoricide'], broad_sel=[False, False, True, True, True, True, False, True, True, True, False, True, False, False, True, False, True, False, False, True, False, True, False, False, False, True, False, False, True, False, True, True, False, False, False, False, False, True, True, False], narrow_words=['belly', 'wallet', 'meadow', 'dairy', 'clap', 'hollow', 'stove', 'forbid', 'warranty', 'contempt', 'stern', 'alley', 'ascertain', 'junk', 'gleam', 'shovel', 'grumble', 'ruthless', 'blink', 'bait', 'tinker', 'dizzy', 'dangle', 'derelict', 'outright', 'wriggle', 'overt', 'glide', 'ledge', 'wag', 'loot', 'seep',